Uma query é um texto!!!!!!!!!

*   Executa/commit
*   Executa/fetch(resgate)



Importando a Biblioteca e conectando com o BD

In [9]:
import sqlite3
connection = sqlite3.connect("pizza_app.sqlite")

Tratamento de erros e atualização acerca da execução de querys

In [10]:
from sqlite3 import Error
def execute_query(connection, query):
  cursor = connection.cursor()
  try:
    cursor.execute(query)
    connection.commit()
    print("Query executada!") # chegou aqui? mudou o bd
    if cursor.rowcount != -1:
      print(f"{cursor.rowcount} linha(s) afetadas")

  except Error as e:
      print(f"Erro: '{e}'")

Criação de tabelas

In [11]:
# Cria a tabela produto #
create_produto_table = \
"""CREATE TABLE produto (
    id_produto INTEGER PRIMARY KEY AUTOINCREMENT,
    tipo VARCHAR(50),
    desc_item VARCHAR(100),
    vl_preco DECIMAL(10, 2)
);"""

execute_query(connection, create_produto_table)
#########################

# Cria a tabela pedido #
create_pedido_table = \
"""CREATE TABLE pedido (
    id_pedido INTEGER PRIMARY KEY AUTOINCREMENT,
    dt_pedido DATE,
    fl_ketchup BOOLEAN,
    desc_uf CHAR(2),
    txt_recado TEXT
);"""

execute_query(connection, create_pedido_table)
#########################

# Cria a tabela item_pedido #
create_item_pedido_table = \
"""CREATE TABLE item_pedido (
    id_pedido INT NOT NULL,
    id_produto INT NOT NULL,
    quantidade INT NOT NULL,
    PRIMARY KEY (id_pedido, id_produto),
    FOREIGN KEY (id_pedido) REFERENCES pedido(id_pedido),
    FOREIGN KEY (id_produto) REFERENCES produto(id_produto)
);"""
execute_query(connection, create_item_pedido_table)
#########################

Erro: 'table produto already exists'
Erro: 'table pedido already exists'
Erro: 'table item_pedido already exists'


Alocando valores hipotéticos nas tabelas para consultas

In [12]:
## Inserindo registros manualmente

# Inserindo produto #
insert_produto = \
"""INSERT INTO
produto (tipo, desc_item, vl_preco)
VALUES
('ingrediente', 'camarão', 6),
('massa', 'tradicional', 9.25),
('borda', 'tradicional', 0),
('queijo', 'muçarela', 4),
('bebida', 'refrigerante', 5);
"""
execute_query(connection, insert_produto)
######################

# Inserindo pedido
insert_pedido = \
"""INSERT INTO
pedido (dt_pedido, fl_ketchup, desc_uf, txt_recado)
VALUES
('2023-06-01', TRUE, 'MG', 'Capricha no queijo!');
"""
execute_query(connection, insert_pedido)
######################

Query executada!
5 linha(s) afetadas
Query executada!
1 linha(s) afetadas


Alocando dados na lista de itens do pedido

In [13]:
# Inserindo item_pedido
itens = (
    {'id_pedido': 1, 'id_produto': 2, 'qtd': 1},
    {'id_pedido': 1, 'id_produto': 3, 'qtd': 1},
    {'id_pedido': 1, 'id_produto': 1, 'qtd': 1},
    {'id_pedido': 1, 'id_produto': 4, 'qtd': 2},
    {'id_pedido': 1, 'id_produto': 5, 'qtd': 3}
)

insert_item_pedido = \
"""INSERT INTO item_pedido (id_pedido, id_produto, quantidade)
VALUES(:id_pedido, :id_produto, :qtd);"""

cursor = connection.cursor()
cursor.executemany(insert_item_pedido, itens)
connection.commit() # necessário para inserções
cursor.close()

Função responsável por ler consultas e exibir mensagens(bem sucedido x erro(e qual erro é))

In [14]:
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()

        return result
    except Error as e:
        print(f"Erro: '{e}'")

Ler a solicitação de informações provindas da tabela produto, exibindo-as

In [15]:
tabela = 'produto'
query = f"SELECT * FROM {tabela}"
resultado = execute_read_query(connection, query)

print(f"Tabela: {tabela}")
for res in resultado:
    print(res)

Tabela: produto
(6, 'ingrediente', 'camarão', 6)
(7, 'massa', 'tradicional', 9.25)
(8, 'borda', 'tradicional', 0)
(9, 'queijo', 'muçarela', 4)
(10, 'bebida', 'refrigerante', 5)


Deletando o que outrora foi inserido manualmente

In [16]:
execute_query(connection, "DELETE FROM item_pedido;")
execute_query(connection, "DELETE FROM pedido;")
execute_query(connection, "DELETE FROM produto;")

Query executada!
5 linha(s) afetadas
Query executada!
1 linha(s) afetadas
Query executada!
5 linha(s) afetadas


# **Introdução ao Pandas**

Baixando...

In [17]:
! wget https://raw.githubusercontent.com/camilalaranjeira/python-intermediario/main/pizza_query/item_pedido.csv
! wget https://raw.githubusercontent.com/camilalaranjeira/python-intermediario/main/pizza_query/pedido.csv
! wget https://raw.githubusercontent.com/camilalaranjeira/python-intermediario/main/pizza_query/produto.csv

--2025-07-16 13:14:54--  https://raw.githubusercontent.com/camilalaranjeira/python-intermediario/main/pizza_query/item_pedido.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 103557 (101K) [text/plain]
Saving to: ‘item_pedido.csv’

item_pedido.csv     100%[===================>] 101.13K  --.-KB/s    in 0.01s   

2025-07-16 13:14:54 (10.3 MB/s) - ‘item_pedido.csv’ saved [103557/103557]

--2025-07-16 13:14:54--  https://raw.githubusercontent.com/camilalaranjeira/python-intermediario/main/pizza_query/pedido.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting

Importação do Pandas

In [18]:
import pandas as pd
df_pedido = pd.read_csv(f'pedido.csv')
display(df_pedido.head())

,id_pedido,dt_pedido,fl_ketchup,desc_uf,txt_recado
0,0,2023-05-11,NaN,GO,NaN
1,1,2023-05-11,NaN,PR,Aquela pizza perfeita! :-D
2,2,2023-05-11,NaN,SP,Muito obrigado!!
3,3,2023-05-11,NaN,SP,NaN
4,4,2023-05-11,NaN,RS,Capricha no peperoni


Introduzindo o método .to_sql: preenchendo a tabela 'pedido'

In [20]:
df_pedido.to_sql('pedido', connection, if_exists='append', index=False)

count_rows = "SELECT COUNT(id_pedido) as count_id FROM pedido;"
print(execute_read_query(connection, count_rows))

select_all = f"SELECT * FROM pedido WHERE id_pedido < 5;"
execute_read_query(connection, select_all)

[(1106,)]


[(0, '2023-05-11', None, 'GO', None),
 (1, '2023-05-11', None, 'PR', 'Aquela pizza perfeita! :-D'),
 (2, '2023-05-11', None, 'SP', 'Muito obrigado!!'),
 (3, '2023-05-11', None, 'SP', None),
 (4, '2023-05-11', None, 'RS', 'Capricha no peperoni')]

Criando a mesma tabela de modo genérico

In [22]:
res = execute_read_query(connection, "SELECT sql FROM sqlite_schema")
for r in res:
    print(r[0])

CREATE TABLE produto (
    id_produto INTEGER PRIMARY KEY AUTOINCREMENT,
    tipo VARCHAR(50),
    desc_item VARCHAR(100),
    vl_preco DECIMAL(10, 2)
)
CREATE TABLE sqlite_sequence(name,seq)
CREATE TABLE pedido (
    id_pedido INTEGER PRIMARY KEY AUTOINCREMENT,
    dt_pedido DATE,
    fl_ketchup BOOLEAN,
    desc_uf CHAR(2),
    txt_recado TEXT
)
CREATE TABLE item_pedido (
    id_pedido INT NOT NULL,
    id_produto INT NOT NULL,
    quantidade INT NOT NULL,
    PRIMARY KEY (id_pedido, id_produto),
    FOREIGN KEY (id_pedido) REFERENCES pedido(id_pedido),
    FOREIGN KEY (id_produto) REFERENCES produto(id_produto)
)
None


""

In [23]:
df_pedido.to_sql('pedido', connection, if_exists='replace', index=False)
res = execute_read_query(connection, "SELECT sql FROM sqlite_schema")
for r in res:
    print(r[0])

CREATE TABLE produto (
    id_produto INTEGER PRIMARY KEY AUTOINCREMENT,
    tipo VARCHAR(50),
    desc_item VARCHAR(100),
    vl_preco DECIMAL(10, 2)
)
CREATE TABLE sqlite_sequence(name,seq)
CREATE TABLE item_pedido (
    id_pedido INT NOT NULL,
    id_produto INT NOT NULL,
    quantidade INT NOT NULL,
    PRIMARY KEY (id_pedido, id_produto),
    FOREIGN KEY (id_pedido) REFERENCES pedido(id_pedido),
    FOREIGN KEY (id_produto) REFERENCES produto(id_produto)
)
None
CREATE TABLE "pedido" (
"id_pedido" INTEGER,
  "dt_pedido" TEXT,
  "fl_ketchup" INTEGER,
  "desc_uf" TEXT,
  "txt_recado" TEXT
)


Finalizando Pandas - limitando a impressão as cinco primeiras linhas da tabela

In [24]:
query="""
SELECT desc_uf, COUNT(*) as count_pedidos
FROM pedido
GROUP BY desc_uf
ORDER BY count_pedidos DESC
LIMIT 5
"""
pd.read_sql_query(query, connection)

,desc_uf,count_pedidos
0,SP,395
1,RJ,103
2,MG,92
3,PR,76
4,RS,50
